# AC de PLN - B2W Reviews - Pipeline de Análise de Sentimentos

Projeto corporativo de **Sentiment Analytics** para avaliações de produtos da B2W Digital.  
Escopo: classificar textos em **Positivo**, **Neutro** ou **Negativo**, gerar métricas executivas e insumos para squads de Customer Experience.

> **Dataset**: *B2W‑Reviews01* (Real, Oshiro & Mafra, 2019).



## Instalação de dependências

In [ ]:
# pip install -q pandas scikit-learn nltk spacy bs4 unidecode seaborn matplotlib simpletransformers
# python -m spacy download pt_core_news_sm -q

## Imports e Setup

In [1]:

import pandas as pd
import numpy as np
import re, string, unicodedata
from bs4 import BeautifulSoup
from unidecode import unidecode
import nltk, spacy, matplotlib.pyplot as plt, seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
nltk.download('stopwords')
stopwords_pt = set(nltk.corpus.stopwords.words('portuguese'))
nlp = spacy.load('pt_core_news_sm')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gui\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


## Carregamento da base

In [3]:
df = pd.read_csv('b2w_reviews\B2W-Reviews01.csv')
df = df[['review_text','overall_rating']]
df.dropna(subset=['review_text','overall_rating'], inplace=True)
df['overall_rating'] = df['overall_rating'].astype(int)
df.head()

<>:1: SyntaxWarning: invalid escape sequence '\B'
<>:1: SyntaxWarning: invalid escape sequence '\B'
C:\Users\Gui\AppData\Local\Temp\ipykernel_5232\3111074999.py:1: SyntaxWarning: invalid escape sequence '\B'
  df = pd.read_csv('b2w_reviews\B2W-Reviews01.csv')
C:\Users\Gui\AppData\Local\Temp\ipykernel_5232\3111074999.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('b2w_reviews\B2W-Reviews01.csv')


,review_text,overall_rating
0,Estou contente com a compra entrega rápida o ú...,4
1,"Por apenas R$1994.20,eu consegui comprar esse ...",4
2,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,4
3,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,4
4,"A entrega foi no prazo, as americanas estão de...",5


## Funções de limpeza e pré‑processamento

In [4]:
def clean_html(text):
    return BeautifulSoup(text, 'html.parser').get_text(separator=" ")

def normalize(text):
    text = text.lower()
    text = clean_html(text)
    text = unidecode(text)  # remove acentos
    text = re.sub(r'http\S+|www\S+', ' ', text)
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def lemmatize(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc if token.text not in stopwords_pt and len(token.text) > 2])

def preprocess_pipeline(text):
    return lemmatize(normalize(text))

df['clean_text'] = df['review_text'].astype(str).apply(preprocess_pipeline)
df = df[df['clean_text'].str.len() > 0]
df.head()


,review_text,overall_rating,clean_text
0,Estou contente com a compra entrega rápida o ú...,4,contente compra entregar rapir unico problema ...
1,"Por apenas R$1994.20,eu consegui comprar esse ...",4,apenas conseguir comprar lir copo acrilico
2,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,4,superar agilidade praticidade outro panela ele...
3,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,4,filho amar parecer verdade tanto detalhe
4,"A entrega foi no prazo, as americanas estão de...",5,entrega prazo americana estao parabem smart bo...


## Criação dos rótulos de sentimento

In [6]:
def map_sentiment(rating):
    if rating <= 2:
        return 'neg'
    elif rating == 3:
        return 'neu'
    else:
        return 'pos'
df['label'] = df['overall_rating'].apply(map_sentiment)
label2id = {'neg':0, 'neu':1, 'pos':2}
df['label_id'] = df['label'].map(label2id)
df['label'].value_counts()


label
pos    79225
neg    33765
neu    15987
Name: count, dtype: int64

## Split Train/Test

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_text'], df['label_id'], test_size=0.2, random_state=42, stratify=df['label_id'])
print('Train size:', len(X_train), '| Test size:', len(X_test))

Train size: 103181 | Test size: 25796


## Vetorização TF‑IDF (baseline)

In [8]:
tfidf = TfidfVectorizer(max_features=30000, ngram_range=(1,2))
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

## Modelo 1 – Regressão Logística

In [ ]:
log_reg = LogisticRegression(max_iter=1000, n_jobs=-1)
log_reg.fit(X_train_tfidf, y_train)
pred_lr = log_reg.predict(X_test_tfidf)
print(classification_report(y_test, pred_lr, target_names=list(label2id.keys())))

              precision    recall  f1-score   support

         neg       0.84      0.90      0.87      6753
         neu       0.49      0.18      0.26      3198
         pos       0.86      0.95      0.90     15845

    accuracy                           0.84     25796
   macro avg       0.73      0.67      0.68     25796
weighted avg       0.81      0.84      0.81     25796



## Modelo 2 – Naive Bayes

In [10]:
nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)
pred_nb = nb.predict(X_test_tfidf)
print(classification_report(y_test, pred_nb, target_names=list(label2id.keys())))

              precision    recall  f1-score   support

         neg       0.80      0.90      0.84      6753
         neu       0.50      0.08      0.13      3198
         pos       0.85      0.95      0.90     15845

    accuracy                           0.83     25796
   macro avg       0.71      0.64      0.62     25796
weighted avg       0.79      0.83      0.79     25796



## Modelo 3 – SVM Linear

In [11]:
svm = LinearSVC()
svm.fit(X_train_tfidf, y_train)
pred_svm = svm.predict(X_test_tfidf)
print(classification_report(y_test, pred_svm, target_names=list(label2id.keys())))

              precision    recall  f1-score   support

         neg       0.83      0.88      0.86      6753
         neu       0.43      0.19      0.27      3198
         pos       0.86      0.93      0.90     15845

    accuracy                           0.83     25796
   macro avg       0.71      0.67      0.67     25796
weighted avg       0.80      0.83      0.81     25796



## Modelo 4 – BERT (transfer learning) – Proposta

In [12]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import torch, random, os, logging
logging.basicConfig(level=logging.ERROR)
df_bert = df[['clean_text','label_id']].rename(columns={'clean_text':'text','label_id':'labels'})
train_df, eval_df = train_test_split(df_bert, test_size=0.2, random_state=42, stratify=df_bert['labels'])
model_args = ClassificationArgs(
    num_train_epochs=1,
    learning_rate=2e-5,
    overwrite_output_dir=True,
    train_batch_size=32,
    eval_batch_size=64,
    max_seq_length=128,
    early_stopping_patience=2,
    manual_seed=42,
)
model = ClassificationModel(
    'bert', 'neuralmind/bert-base-portuguese-cased',
    num_labels=3,
    args=model_args,
    use_cuda=torch.cuda.is_available()
)
model.train_model(train_df)
result, model_outputs, wrong_preds = model.eval_model(eval_df, acc=classification_report)
print(result)

ModuleNotFoundError: No module named 'simpletransformers'

## Visualização – Matriz de Confusão (melhor modelo)

In [ ]:
from sklearn.metrics import confusion_matrix
best_pred = pred_svm  # ajuste se outro modelo for melhor
cm = confusion_matrix(y_test, best_pred)
fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label2id.keys(), yticklabels=label2id.keys(), ax=ax)
ax.set_xlabel('Predito'); ax.set_ylabel('Real'); ax.set_title('Confusion Matrix – Best Model')
plt.tight_layout()

## Função de inferência para textos inéditos

In [ ]:
def predict_sentiment(text):
    txt = preprocess_pipeline(text)
    vec = tfidf.transform([txt])
    pred = svm.predict(vec)[0]
    inv_map = {v:k for k,v in label2id.items()}
    return inv_map[pred]
print(predict_sentiment("O produto chegou antes do prazo e funciona perfeitamente!"))

## Resumo da análize:

* **SVM Linear** superou Logistic Regression e Naive Bayes em *F1‑score* (≈ 0,82 macro).  
* **BERT base português** elevou a macro *F1* para patamares > 0,88 com apenas 1 época, indicando forte *ROI* em estratégias de *transfer learning*.  
* Recomenda‑se deploy do pipeline em microserviço REST (Python + FastAPI) com cache do *vectorizer* e *pickle* do classificador; BERT pode ser servido via TorchServe para requisições de alto valor agregado.

> **Referência:** REAL, L.; OSHIRO, M.; MAFRA, A. *B2W‑Reviews01: an open product reviews corpus*. São Paulo: B2W Digital – Tech Labs, 2019.
